In [1]:
import pysolr 

#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/collection/', timeout=10)

In [3]:
r = solr.search(q="id:bkc-6", fl="*,[child]")
[doc] = r.docs
doc

{'id': 'bkc-6',
 'type': ['Item'],
 'heldBy': ['Biblioteca do INPA'],
 'cdd': [546.5],
 'cutter': ['O48a'],
 'year': [2023],
 'shelf': ['E1-P1'],
 'barcode': ['23-0004'],
 '_version_': 1772331636282621952,
 'itemOf': {'id': 'bkc-6/itemOf/bkc-2',
  'mainTitle': ['Physiological plant ecology'],
  'uri': ['https://bibliokeia.com/resources/instance/bkc-2'],
  '_version_': 1772331636282621952}}

In [4]:
itemOf = doc['itemOf']['id']
itemOf

'bkc-6/itemOf/bkc-2'

In [10]:
solr.delete(id='bkc-2/hasItem/bkc-6')

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">1</int>\n</lst>\n</response>\n'

In [6]:
x = solr.delete(id=['bkc-6', "bkc-2/hasItem/bkc-6", itemOf], commit=True)

x

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">65</int>\n</lst>\n</response>\n'